In [1]:
from __future__ import division
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import scipy.optimize as op
import xgboost as xgb
from sklearn.metrics import mean_squared_error,make_scorer,accuracy_score,recall_score,precision_score,roc_auc_score
from sklearn.model_selection import cross_val_score, KFold, cross_validate,StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit 
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics as metrices
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from statsmodels.stats.proportion import proportion_confint

%matplotlib inline

pd.options.display.float_format = '{:.3f}'.format
%config IPCompleter.greedy=True
import scipy as sc
import colorama
from colorama import Fore, Style

In [10]:
afib_cohort = pd.read_csv('afib_diagnosis_pandas.csv',low_memory=False)

In [11]:
afib_diagnosis_pandas_limited = afib_cohort[['AGE_AT_INDEX','PT_SEX_CODE','FAM_HEART_HIST','FAM_STROKE_HIST','HYPERTENSION_AT_INDEX',
                                                               'MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX',
                                                               'HYPERCOAG_STATES_ALL_TIME','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX',
                                                               'CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX',
                                                               'CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX',
                                                               'PFO_ALL_TIME','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX',
                                                               'BMI_CLOSEST_TO_INDEX','HB_VALUE','PLT_VALUE','OBESITY_AT_INDEX','CAD_AT_INDEX','VHD_AT_INDEX','AFIB_DIAGNOSED']]  

In [4]:
def chi_square(df,col_name, prob = 0.95, alpha = 0.05):
    flag = True
    cross= pd.crosstab(df["AFIB_DIAGNOSED"], df[col_name])
    cross_margin= pd.crosstab(df["AFIB_DIAGNOSED"], df[col_name],margins=True)
    chi,p,dof,exp_freq= sc.stats.chi2_contingency(cross)
    critical = sc.stats.chi2.ppf(prob, dof)
    print(Fore.BLUE + '\n\n------'+ col_name + '-----')
    print(Style.RESET_ALL)
    print('\n', cross_margin)
    print('\n', 100*cross_margin.iloc[:,:].div(cross_margin.All, axis=0))
    print('\n====================\n')
    print('chi square statistics: ', chi)
    print('critical value: ', critical)
    print('p-value: ', p)
    print('dof: ', dof)
    if chi >= critical or p <= alpha :
        print(f"{Fore.GREEN}Outcome: Significant{Style.RESET_ALL}")
    else:
        print(f"{Fore.RED}Outcome: Non significant{Style.RESET_ALL}")
        flag = False
    print('====================')
    return flag
    

In [5]:
def non_parametric_test(df,grp1,grp2,col_name, alpha = 0.05):
    flag = True
    a = df[df['AFIB_DIAGNOSED'] == grp1][col_name].dropna()
    b = df[df['AFIB_DIAGNOSED'] == grp2][col_name].dropna()
    stat, p = sc.stats.ranksums(a, b)
    print(Fore.BLUE + '\n\n------'+ col_name + '-----')
    print(Style.RESET_ALL)
    print('grp:',grp1, 'count: ',a.count())
    print('grp:',grp1, 'std: ',a.std())
    print('grp:',grp1, 'mean: ',a.mean())
    print('grp',grp1, 'median',a.quantile(0.5))
    print('grp',grp1, 'IQR',a.quantile(0.75) - a.quantile(0.25))
    print('grp',grp1, '25%',a.quantile(0.25))
    print('grp',grp1, '75%',a.quantile(0.75))
    print('grp:',grp2, 'count: ',b.count())
    print('grp:',grp2, 'std: ',b.std())
    print('grp:',grp2, 'mean: ',b.mean())
    print('grp',grp2, 'median',b.quantile(0.5))
    print('grp',grp2, 'IQR',b.quantile(0.75) - b.quantile(0.25))
    print('grp',grp1, '25%',b.quantile(0.25))
    print('grp',grp1, '75%',b.quantile(0.75))
    print('\n')
    print('wilcoxon statistics: ', stat)
    print('p-value: ', p)
    if  p <= alpha :
        print(f"{Fore.GREEN}Outcome: Significant{Style.RESET_ALL}")
    else:
        print(f"{Fore.RED}Outcome: Non significant{Style.RESET_ALL}")
        flag = False
    print('====================')
    return flag

In [6]:
Categorical_variables_demograph=[
'PT_SEX_CODE',
'FAM_HEART_HIST',
'FAM_STROKE_HIST'
]

Categorical_variables_comorbidities=[
'HYPERTENSION_AT_INDEX',
'MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX',
'HYPERCOAG_STATES_ALL_TIME','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX',
'CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX',
'CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX',
'PFO_ALL_TIME','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX',
'OBESITY_AT_INDEX','CAD_AT_INDEX','VHD_AT_INDEX'
]

Continuous_vars=[
'AGE_AT_INDEX',
'BMI_CLOSEST_TO_INDEX',
'HB_VALUE',
'PLT_VALUE'
]

categorical_vars = Categorical_variables_demograph + Categorical_variables_comorbidities 
                 

In [8]:
from statsmodels.imputation import mice 
imp = mice.MICEData(afib_diagnosis_pandas_limited)
imp.update_all(10)
afib_diagnosis_pandas_limited = imp.data.copy()

In [ ]:
sig_count = 0
non_sig_count = 0
sig_vars = []
for attr in categorical_vars:
    flag = chi_square(afib_diagnosis_pandas_limited,attr)
    if flag:
        sig_count+=1
        sig_vars.append(attr)
    else:
        non_sig_count+=1

print('=============Overall Stats==================')
print(Fore.GREEN +'Significant vars count: ',sig_count)
print(Style.RESET_ALL)
print(Fore.RED +'Non significant vars count: ' ,non_sig_count)
print(Style.RESET_ALL)
print(sig_vars)



------PT_SEX_CODE-----


 PT_SEX_CODE       0    1   All
AFIB_DIAGNOSED                
0               617  692  1309
1               302  281   583
All             919  973  1892

 PT_SEX_CODE         0      1     All
AFIB_DIAGNOSED                      
0              47.135 52.865 100.000
1              51.801 48.199 100.000
All            48.573 51.427 100.000


chi square statistics:  3.3309395852331773
critical value:  3.841458820694124
p-value:  0.06798802449732873
dof:  1
Outcome: Non significant


------FAM_HEART_HIST-----


 FAM_HEART_HIST     0    1   All
AFIB_DIAGNOSED                 
0                711  598  1309
1                370  213   583
All             1081  811  1892

 FAM_HEART_HIST      0      1     All
AFIB_DIAGNOSED                      
0              54.316 45.684 100.000
1              63.465 36.535 100.000
All            57.135 42.865 100.000


chi square statistics:  13.413408791067745
critical value:  3.841458820694124
p-value:  0.00024983199800702

In [12]:
sig_count = 0
non_sig_count = 0
sig_vars=[]
for attr in Continuous_vars:
    flag = non_parametric_test(afib_diagnosis_pandas_limited,0,1,attr)
    if flag:
        sig_count+=1
        sig_vars.append(attr)
    else:
        non_sig_count+=1

print('=============Overall Stats==================')
print(Fore.GREEN +'Significant vars count: ',sig_count)
print(Style.RESET_ALL)
print(Fore.RED +'Non significant vars count: ' ,non_sig_count)
print(Style.RESET_ALL)
print(sig_vars)



------AGE_AT_INDEX-----

grp: 0 count:  1309
grp: 0 std:  14.193026018341715
grp: 0 mean:  62.64499618029029
grp 0 median 64.1
grp 0 IQR 19.299999999999997
grp 0 25% 53.8
grp 0 75% 73.1
grp: 1 count:  583
grp: 1 std:  11.06626606320249
grp: 1 mean:  74.85437392795885
grp 1 median 76.7
grp 1 IQR 15.450000000000003
grp 0 25% 68.35
grp 0 75% 83.8


wilcoxon statistics:  -17.82935245398536
p-value:  4.182370758702976e-71
Outcome: Significant


------BMI_CLOSEST_TO_INDEX-----

grp: 0 count:  1262
grp: 0 std:  6.439813494632132
grp: 0 mean:  29.944770206022188
grp 0 median 29.0
grp 0 IQR 7.774999999999999
grp 0 25% 25.6
grp 0 75% 33.375
grp: 1 count:  569
grp: 1 std:  6.761243792507655
grp: 1 mean:  29.545166959578207
grp 1 median 28.3
grp 1 IQR 8.400000000000002
grp 0 25% 24.8
grp 0 75% 33.2


wilcoxon statistics:  1.7244539336266336
p-value:  0.08462592830917631
Outcome: Non significant


------HB_VALUE-----

grp: 0 count:  1171
grp: 0 std:  1.7080035237056188
grp: 0 mean:  14.0364645602